In [1]:
import tabula as tb
import pandas as pd
import PyPDF2
from PyPDF2 import PdfReader
import re
import docx
from docx import Document
import zipfile
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import os
import pandas as pd
import csv
import json
import re

In [2]:
def filterHTMLstr(str):
    html_tag = {'&#xA;': '\n', '&quot;': '\"', '&amp;': '', '&lt;': '<', '&gt;': '>',
                '&apos;': "'", '&nbsp;': ' ', '&yen;': '¥', '&copy;': '©', '&divide;': '÷'
        , '&times;': 'x', '&trade;': '™', '&reg;': '®', '&sect;': '§', '&euro;': '€',
                '&pound;': '£', '&cent;': '￠', '&raquo;': '»', '&nbsp': ' ', u'\xa0': ' ',
                '\n': ' ', '\t': ' ', '    ': '', '&emsp': ' ', ' ,':',', '  ':' '
                }
    for k, v in html_tag.items():
        str = str.replace(k, v)
        # str = str.replace(k[1:], v)
    str = " ".join(str.split())
    str = str.strip('\n')
    str = str.strip(' ')
    return str

def MultipleFileProcess(files_position):
    totaldata = []
    for position in files_position:
        data = SingleHTMLProcess(position)
        totaldata.append(data)

    return totaldata

def to_CSV(totaldata):
    pd.DataFrame(totaldata).to_csv('./result/2019.csv', index=False, encoding="utf-8")

In [12]:
### code entry
path = "./data/2019.html"
htmlfile = open(path, 'r', encoding='utf-8')
html=htmlfile  
bs = BeautifulSoup(html, "lxml")  # 缩进格式
totaldata = handle(bs)
to_CSV(totaldata)

topic name: Focus Area 1 In-Space Propulsion Technologies
subtopic name: T2.02Advanced Technologies for In-Space Electric Propulsion (EP)
subtopic name: T2.03Performance Demonstration of High Payoff Propulsion Technology: Rotating Detonation Engine and Dual Mode Ionic Liquid
subtopic name: Z10.01Cryogenic Fluid Management
subtopic name: Z10.02In-Space Electric Propulsion Component Technologies
subtopic name: Z10.03Nuclear Thermal Propulsion
topic name: Focus Area 2 Power Energy and Storage
subtopic name: S3.01Power Generation and Conversion
subtopic name: S3.03Power Electronics and Management, and Energy Storage
subtopic name: Z1.03Kilowatt-Class Energy Conversion for Small Fission Reactors
subtopic name: Z1.04Long Duration Lunar Energy Storage and Discharge
topic name: Focus Area 3 Autonomous Systems for Space Exploration
subtopic name: H6.01Integrated Systems Health Management for Sustainable Habitats
subtopic name: H6.03Spacecraft Autonomous Agent Cognitive Architectures for Human E

In [11]:
def handle(bs):
    totalData = []
    ### get overall topic
    evens = bs.find_all("li", {"class": "md-taxonomy even"})
    odds = bs.find_all("li", {"class": "md-taxonomy odd"})
    num_dic = {}
    for index in range(23):
        dic = {}
        topic_block = evens[int(index / 2)] if int(index % 2) == 0 else odds[int(index / 2)]
        dic["Focus Area"] = getTopicName(topic_block)
        ### get sub-focus topics
        subtopic_even = topic_block.find_all("li", {"class": "first-topic even"})
        subtopic_odd = topic_block.find_all("li", {"class": "first-topic odd"})
        for index2 in range(len(subtopic_even) + len(subtopic_odd)):
            subtopic_block = subtopic_even[int(index2 / 2)] if int(index2 % 2) == 0 else subtopic_odd[int(index2 / 2)]
            dic["Subtopic"] = getSubtopicName(subtopic_block)
            metaDic = getMetaData(subtopic_block)
            for k in metaDic:
                dic[k] = metaDic[k]
            ### get other data
            whole_info_block = subtopic_block.find("div", {"class": "whole-topic-content"})
            new_dic = dic.copy()
            new_dic["Content"] = filterHTMLstr(whole_info_block.get_text())
            totalData.append(new_dic)
    return totalData

In [4]:
### get topic and subtopic name
def getTopicName(topic_block):
    topic_name = filterHTMLstr(topic_block.find("h2").get_text())
    print("topic name:", topic_name)  # topic name
    return topic_name

### handle every sub-topic
def getSubtopicName(subtopic_block):
    subtopic_name = filterHTMLstr(subtopic_block.find("p", {"class": "subtopic"}).get_text())
    print("subtopic name:", subtopic_name)  # subtopic name
    return subtopic_name

In [5]:
### get Lead Center, Participating Center(s), Solicitation Year
def getMetaData(subtopic_block):
    dic = {}
    info_group = subtopic_block.find_all("p", {"class": "l-center"})
    for raw_info in info_group:
        info = filterHTMLstr(raw_info.get_text())
        pair = info.split(":")
        dic[pair[0]] = pair[1]
        # print(pair[0], pair[1])
    # print(dic)
    return dic